<a href="https://colab.research.google.com/github/emrecan1923/yapayzekaprojelerim/blob/main/Hisse_Senedi_Tahmini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Verileri almak için hisse senedi kodunu ve tarih aralığını belirleyin
hisse_kodu = 'NETAS.IS'  # Netaş'ın BIST kodu
veri_baslangic = '2022-01-01'  # Son 1 yılın verilerini kullan

# Hisse senedi verilerini alın
veriler = yf.download(hisse_kodu, start=veri_baslangic, progress=False)

# Kapanış fiyatlarını alın
kapanis_fiyatlari = veriler['Close'].values.reshape(-1, 1)

# Verileri normalize etmek için MinMaxScaler kullanın
scaler = MinMaxScaler(feature_range=(0, 1))
kapanis_fiyatlari_olcekli = scaler.fit_transform(kapanis_fiyatlari)

# Zaman serisi pencelerini oluşturun (10 günlük pencere boyutu)
pencere_boyutu = 10

def zaman_serisi_pencereleri_olustur(veri, pencere_boyutu):
    x = []
    y = []
    for i in range(len(veri) - pencere_boyutu - 1):
        x.append(veri[i:(i + pencere_boyutu), 0])
        y.append(veri[i + pencere_boyutu, 0])
    return np.array(x), np.array(y)

# Verileri eğitim ve test olarak bölelim
egitim_orani = 0.8
egitim_veri_sayisi = int(len(kapanis_fiyatlari_olcekli) * egitim_orani)

egitim_verisi = kapanis_fiyatlari_olcekli[:egitim_veri_sayisi]
test_verisi = kapanis_fiyatlari_olcekli[egitim_veri_sayisi:]

egitim_x, egitim_y = zaman_serisi_pencereleri_olustur(egitim_verisi, pencere_boyutu)
test_x, test_y = zaman_serisi_pencereleri_olustur(test_verisi, pencere_boyutu)

# LSTM modelini oluşturun
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(pencere_boyutu, 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Modeli eğit
model.fit(egitim_x, egitim_y, epochs=50, batch_size=32)

# Bugünkü fiyatı tahmin etmek için kullanılacak veri
bugunku_veri = kapanis_fiyatlari_olcekli[-pencere_boyutu:]

# Tahminleri yap
tahminler = []
for i in range(1):  # Yarın için tahmin yapacak
    tahmin = model.predict(bugunku_veri.reshape(1, pencere_boyutu, 1))
    tahminler.append(tahmin[0, 0])  # Tahminleri listeye ekle
    bugunku_veri = np.append(bugunku_veri[1:], tahmin)  # Yeni tahmini veriye ekle

# Gerçek veriler ile karşılaştırmak için tahminlerin ölçeklemesini geri dönüştür
tahminler = np.array(tahminler).reshape(-1, 1)
tahminler_gercek_skala = scaler.inverse_transform(tahminler)

# Bugünkü fiyat ve bir önceki günün fiyatını gerçek verilerden alın
bugunku_veriler = yf.download(hisse_kodu, period="2d")  # Bugünkü ve bir önceki gün
bugunku_kapanis_fiyati = bugunku_veriler['Close'].iloc[-1]

gercek_test_veriler = yf.download(hisse_kodu, start=veri_baslangic, end=bugunku_veriler.index[-2])

# Gerçek test verilerinden kapanış fiyatlarını alın
gercek_test_kapanis_fiyatlari = gercek_test_veriler['Close'].values.reshape(-1, 1)

# Gerçek test kapanış fiyatlarını oluştur
gercek_test_y = gercek_test_kapanis_fiyatlari[-2:]

print(f"Bugünkü Fiyat: {bugunku_kapanis_fiyati}")
# Tahminleri yazdır
print(f"Tahminler: {tahminler_gercek_skala}")



Epoch 1/50
13/13 [==============================] - 9s 19ms/step - loss: 0.0295
Epoch 2/50
13/13 [==============================] - 0s 21ms/step - loss: 0.0083
Epoch 3/50
13/13 [==============================] - 0s 20ms/step - loss: 0.0048
Epoch 4/50
13/13 [==============================] - 0s 20ms/step - loss: 0.0036
Epoch 5/50
13/13 [==============================] - 0s 28ms/step - loss: 0.0036
Epoch 6/50
13/13 [==============================] - 0s 32ms/step - loss: 0.0031
Epoch 7/50
13/13 [==============================] - 0s 20ms/step - loss: 0.0035
Epoch 8/50
13/13 [==============================] - 0s 19ms/step - loss: 0.0040
Epoch 9/50
13/13 [==============================] - 0s 20ms/step - loss: 0.0032
Epoch 10/50
13/13 [==============================] - 0s 19ms/step - loss: 0.0031
Epoch 11/50
13/13 [==============================] - 0s 19ms/step - loss: 0.0032
Epoch 12/50
13/13 [==============================] - 0s 19ms/step - loss: 0.0031
Epoch 13/50
13/13 [==================

Veri Alımı: yfinance kütüphanesi aracılığıyla belirtilen hisse senedi kodundan ve tarih aralığından (1 yıllık veri) hisse senedi verilerini alır.

Veri Ön İşleme: Alınan kapanış fiyatları, veri normalizasyonu için MinMaxScaler kullanılarak ölçeklenir.

Zaman Serisi Pencereleri Oluşturma: Belirli bir pencere boyutuna göre veriler, zaman serisi pencereciklerine bölünür. Bu pencerecikler, girdi ve çıktı verileri olarak ayrılır. Burada LSTM modeline veri sağlamak için kullanılacak format oluşturulur.

Eğitim ve Test Verisi Ayrımı: Veriler, belirli bir oranda (örneğin, %80) eğitim ve geri kalan kısmı test verisi olarak ayrılır.

LSTM Modeli Oluşturma: Birkaç LSTM katmanı, dropout katmanları ve yoğun (Dense) katmanlar ile bir sinir ağı modeli oluşturulur. Bu modelin derlenmesi ve eğitilmesi gerçekleştirilir.

Tahminlerin Yapılması: Eğitilen model, test verisi üzerinde tahminler yapar. Bugünkü fiyatı tahmin etmek için en son veriler kullanılır. İki gün için tahmin yapılır.

Gerçek Verilerle Karşılaştırma: Gerçek verilerle yapılan tahminler arasındaki farklılıklar, ölçeklendirme işlemi kullanılarak hesaplanır.
Modelin Kaydedilmesi: Eğitilen model, belirtilen dosya adıyla kaydedilir (hisseyi_tahmin_et_modeli.h5).

In [ ]:
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

# Hisseden veri al
hisse_kodu = 'NETAS.IS'  # Netaş'ın BIST kodu
veri_baslangic = '2022-01-01'  # Son 1 yılın verilerini kullan

veriler = yf.download(hisse_kodu, start=veri_baslangic, progress=False)
kapanis_fiyatlari = veriler['Close'].values.reshape(-1, 1)

# Verileri normalize et
scaler = MinMaxScaler(feature_range=(0, 1))
kapanis_fiyatlari_olcekli = scaler.fit_transform(kapanis_fiyatlari)

# Modeli yükle
model = load_model("/content/drive/MyDrive/hisseyi_tahmin_et_modeli.h5")

# Son 10 günü kullanarak tahmin yap
pencere_boyutu = 10
son_veri = kapanis_fiyatlari_olcekli[-pencere_boyutu:]
son_veri = son_veri.reshape((1, pencere_boyutu, 1))

tahmin = model.predict(son_veri)
tahmin = scaler.inverse_transform(tahmin)

bugunku_veri = yf.download(hisse_kodu, period="1d")
bugunku_kapanis_fiyati = bugunku_veri['Close'].iloc[-1]

print(f"Gelecek Gün Tahmini: {tahmin[0, 0]}")
print(f"Bugünkü Fiyat: {bugunku_kapanis_fiyati}")


1/1 [==============================] - 2s 2s/step
[*********************100%%**********************]  1 of 1 completed
Gelecek Gün Tahmini: 36.2808723449707
Bugünkü Fiyat: 38.459999084472656


yfinance kütüphanesi aracılığıyla belirli bir hisse senedinin (burada NETAS.IS - Netaş'ın BIST kodu) fiyat verilerini çeker.
Hisse senedi kapanış fiyatlarını alır ve bu verileri kullanarak bir zaman serisi tahmini yapmak için bir yapay sinir ağı modeli kullanır.
Verileri normalize eder. Normalizasyon, verilerin belirli bir aralığa ölçeklenmesini sağlar (burada 0 ile 1 arasında).
Önceden eğitilmiş bir yapay sinir ağı modelini (hisseyi_tahmin_et_modeli.h5) yükler.
Son 10 günün kapanış fiyatları verisi üzerinden gelecek bir günün fiyatını tahmin eder.
Tahmin edilen fiyatı geri dönüştürerek, ölçeklendirme işlemi öncesindeki orijinal değerine dönüştürür.
Güncel hisse senedi fiyatını çeker (yfinance kullanarak).
Tahmin edilen fiyatı ve güncel fiyatı ekrana yazdırır.

In [ ]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:          6
    Model:               79
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4400.33
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clf
                         lush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_
                         good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fm
                         a cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyp
                         ervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd i